In [ ]:

%pip install timm albumentations -q


In [ ]:
import os
import time
import random
import warnings
warnings.filterwarnings('ignore')

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight



In [ ]:
# 시드 고정
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False



## Config (EDA 기반 최적화)


In [ ]:
CFG = {
    # 데이터
    'IMG_SIZE': 384,
    'BATCH_SIZE': 8,
    'NUM_CLASSES': 17,
    
    # 학습
    'EPOCHS': 15,              # 빠른 실험
    'LR': 2.5e-5,
    'WEIGHT_DECAY': 1e-5,
    
    # 모델
    'MODEL_NAME': 'efficientnet_b4',  # 검증됨
    
    # 경로
    'BASE_DIR': '/home/realtheai/cv_competetion',
    'TRAIN_CSV': '/home/realtheai/cv_competetion/data/train.csv',
    'TEST_CSV': '/home/realtheai/cv_competetion/data/sample_submission.csv',
    'TRAIN_IMG_DIR': '/home/realtheai/cv_competetion/data/train',
    'TEST_IMG_DIR': '/home/realtheai/cv_competetion/data/test',
    
    # 디바이스
    'DEVICE': 'cuda' if torch.cuda.is_available() else 'cpu',
    
    # Augmentation (EDA 기반!)
    'USE_CLASS_WEIGHTS': True,
}

print("\n📊 Configuration")
print("="*60)
print(f"모델: {CFG['MODEL_NAME']}")
print(f"IMG_SIZE: {CFG['IMG_SIZE']}")
print(f"BATCH_SIZE: {CFG['BATCH_SIZE']}")
print(f"EPOCHS: {CFG['EPOCHS']}")
print(f"Device: {CFG['DEVICE']}")
print("="*60)



## Dataset & Augmentation


In [ ]:
# Augmentation (EDA 기반 최적화!)
trn_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    
    # 🔥 선명도 차이 대응 (최우선!)
    A.GaussianBlur(blur_limit=(3, 9), p=0.6),  # 강하게!
    A.MotionBlur(blur_limit=7, p=0.3),
    
    # 문서 특화 (회전은 약하게)
    A.Rotate(limit=3, p=0.3),
    A.ShiftScaleRotate(
        shift_limit=0.05,
        scale_limit=0.1,
        rotate_limit=3,
        p=0.3
    ),
    
    # 밝기 (차이 작음, 적당히)
    A.RandomBrightnessContrast(
        brightness_limit=0.3,
        contrast_limit=0.2,
        p=0.5
    ),
    
    # 품질 저하
    A.ImageCompression(quality_lower=60, p=0.3),
    A.Downscale(scale_min=0.75, scale_max=0.95, p=0.2),
    
    # 노이즈
    A.GaussNoise(var_limit=(10, 50), p=0.3),
    
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

tst_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])



In [ ]:
class ImageDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.iloc[idx]['ID'])
        img = Image.open(img_path).convert('RGB')
        img = np.array(img)
        
        if self.transform:
            img = self.transform(image=img)['image']
        
        if 'target' in self.df.columns:
            target = self.df.iloc[idx]['target']
            return img, target
        return img

print("✅ Dataset 클래스 정의 완료")


## Model


In [ ]:
class DocumentClassifier(nn.Module):
    def __init__(self, model_name, num_classes, pretrained=True):
        super().__init__()
        self.backbone = timm.create_model(model_name, pretrained=pretrained, num_classes=0)
        in_features = self.backbone.num_features
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(in_features, num_classes)
        )
    
    def forward(self, x):
        features = self.backbone(x)
        return self.classifier(features)

print("✅ Model 클래스 정의 완료")


## Training Functions


In [8]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    preds_list, targets_list = [], []
    
    for imgs, targets in tqdm(loader, desc="Train"):
        imgs, targets = imgs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        preds_list.extend(outputs.argmax(dim=1).cpu().numpy())
        targets_list.extend(targets.cpu().numpy())
    
    acc = accuracy_score(targets_list, preds_list)
    f1 = f1_score(targets_list, preds_list, average='macro')
    
    return total_loss / len(loader), acc, f1

@torch.no_grad()
def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    preds_list, targets_list = [], []
    
    for imgs, targets in tqdm(loader, desc="Valid"):
        imgs, targets = imgs.to(device), targets.to(device)
        
        outputs = model(imgs)
        loss = criterion(outputs, targets)
        
        total_loss += loss.item()
        preds_list.extend(outputs.argmax(dim=1).cpu().numpy())
        targets_list.extend(targets.cpu().numpy())
    
    acc = accuracy_score(targets_list, preds_list)
    f1 = f1_score(targets_list, preds_list, average='macro')
    
    return total_loss / len(loader), acc, f1, preds_list, targets_list

print("✅ Train/Valid 함수 정의 완료")


✅ Train/Valid 함수 정의 완료


## Data Loading


In [ ]:
# 데이터 로드
train_df = pd.read_csv(CFG['TRAIN_CSV'])

# Train/Valid Split
train_data, valid_data = train_test_split(
    train_df,
    test_size=0.2,
    stratify=train_df['target'],
    random_state=SEED
)

# Dataset & DataLoader
trn_dataset = ImageDataset(CFG['TRAIN_CSV'], CFG['TRAIN_IMG_DIR'], trn_transform)
trn_dataset.df = train_data.reset_index(drop=True)

val_dataset = ImageDataset(CFG['TRAIN_CSV'], CFG['TRAIN_IMG_DIR'], tst_transform)
val_dataset.df = valid_data.reset_index(drop=True)

tst_dataset = ImageDataset(CFG['TEST_CSV'], CFG['TEST_IMG_DIR'], tst_transform)

trn_loader = DataLoader(trn_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=4)
tst_loader = DataLoader(tst_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=4)

print(f"✅ Train: {len(trn_dataset)} | Valid: {len(val_dataset)} | Test: {len(tst_dataset)}")


## Model Setup


In [ ]:
# 모델 초기화
model = DocumentClassifier(CFG['MODEL_NAME'], CFG['NUM_CLASSES']).to(CFG['DEVICE'])

# Class Weights (불균형 대응)
if CFG['USE_CLASS_WEIGHTS']:
    class_weights = compute_class_weight('balanced', classes=np.unique(train_df['target']), y=train_df['target'])
    class_weights = torch.FloatTensor(class_weights).to(CFG['DEVICE'])
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    print("✅ Weighted CrossEntropyLoss")
else:
    criterion = nn.CrossEntropyLoss()
    print("✅ CrossEntropyLoss")

# Optimizer & Scheduler
optimizer = Adam(model.parameters(), lr=CFG['LR'], weight_decay=CFG['WEIGHT_DECAY'])
scheduler = CosineAnnealingLR(optimizer, T_max=CFG['EPOCHS'], eta_min=1e-6)

print(f"✅ Model: {CFG['MODEL_NAME']}")
print(f"✅ Optimizer: Adam (LR={CFG['LR']})")


## Training (15 Epochs)


In [ ]:
print("\n" + "="*80)
print("🚀 Training 시작!")
print("="*80)

best_f1 = 0
best_epoch = 0
history = {'train_loss': [], 'train_f1': [], 'val_loss': [], 'val_f1': []}

for epoch in range(1, CFG['EPOCHS'] + 1):
    print(f"\nEpoch {epoch}/{CFG['EPOCHS']}")
    print("-" * 60)
    
    # Train
    trn_loss, trn_acc, trn_f1 = train_one_epoch(model, trn_loader, criterion, optimizer, CFG['DEVICE'])
    
    # Validate
    val_loss, val_acc, val_f1, val_preds, val_targets = validate(model, val_loader, criterion, CFG['DEVICE'])
    
    # Scheduler
    scheduler.step()
    
    # History
    history['train_loss'].append(trn_loss)
    history['train_f1'].append(trn_f1)
    history['val_loss'].append(val_loss)
    history['val_f1'].append(val_f1)
    
    # 결과 출력
    print(f"Train Loss: {trn_loss:.4f} | F1: {trn_f1:.4f}")
    print(f"Valid Loss: {val_loss:.4f} | F1: {val_f1:.4f}")
    
    # Best 모델 저장
    if val_f1 > best_f1:
        best_f1 = val_f1
        best_epoch = epoch
        torch.save(model.state_dict(), f"{CFG['BASE_DIR']}/06_best_model.pth")
        print(f"✅ Best F1: {best_f1:.4f} (Saved!)")

print("\n" + "="*80)
print(f"✅ Training 완료!")
print(f"   Best F1: {best_f1:.4f} (Epoch {best_epoch})")
print("="*80)


## Results Visualization


In [ ]:
# 학습 곡선
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Valid')
axes[0].set_title('Loss')
axes[0].legend()
axes[0].grid(alpha=0.3)

# F1 Score
axes[1].plot(history['train_f1'], label='Train')
axes[1].plot(history['val_f1'], label='Valid')
axes[1].axhline(best_f1, color='red', linestyle='--', label=f'Best: {best_f1:.4f}')
axes[1].set_title('F1 Score')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(f"{CFG['BASE_DIR']}/training_curves.png", dpi=150, bbox_inches='tight')
plt.show()



## Wrong Predictions Analysis


In [ ]:
def analyze_wrong_predictions(val_df, preds_list, targets_list, img_dir, meta_df=None, save_path=None):
    """틀린 이미지 시각화 및 패턴 분석"""
    # 틀린 인덱스 찾기
    wrong_indices = [i for i, (pred, target) in enumerate(zip(preds_list, targets_list)) if pred != target]
    
    print(f"\n{'='*60}")
    print(f"📊 검증 결과 분석")
    print(f"{'='*60}")
    print(f"   전체: {len(preds_list)}개")
    print(f"   정답: {len(preds_list) - len(wrong_indices)}개")
    print(f"   오답: {len(wrong_indices)}개 ({len(wrong_indices)/len(preds_list)*100:.1f}%)")
    print(f"   정확도: {(len(preds_list) - len(wrong_indices))/len(preds_list)*100:.1f}%")
    
    if len(wrong_indices) == 0:
        print("\n🎉 모든 예측이 정확합니다!")
        return
    
    # 클래스별 오류 분석
    wrong_by_true_class = {}
    for idx in wrong_indices:
        true_class = targets_list[idx]
        if true_class not in wrong_by_true_class:
            wrong_by_true_class[true_class] = []
        wrong_by_true_class[true_class].append(idx)
    
    print(f"\n📈 클래스별 오답 분석:")
    for class_id in sorted(wrong_by_true_class.keys()):
        count = len(wrong_by_true_class[class_id])
        if meta_df is not None:
            class_name = meta_df[meta_df['target'] == class_id]['class_name'].values[0]
            print(f"   Class {class_id:2d} ({class_name[:20]:20s}): {count:2d}개 오답")
        else:
            print(f"   Class {class_id:2d}: {count:2d}개 오답")
    
    # 틀린 이미지 시각화 (12개)
    if save_path:
        n_show = min(12, len(wrong_indices))
        fig, axes = plt.subplots(3, 4, figsize=(16, 12))
        axes = axes.flatten()
        
        for i, idx in enumerate(wrong_indices[:n_show]):
            img_name = val_df.iloc[idx]['ID']
            img_path = os.path.join(img_dir, img_name)
            
            if os.path.exists(img_path):
                img = Image.open(img_path)
                axes[i].imshow(img)
                
                true_label = f"True: {targets_list[idx]}"
                pred_label = f"Pred: {preds_list[idx]}"
                
                if meta_df is not None:
                    true_class = meta_df[meta_df['target'] == targets_list[idx]]['class_name'].values[0]
                    pred_class = meta_df[meta_df['target'] == preds_list[idx]]['class_name'].values[0]
                    true_label += f"\n({true_class[:15]})"
                    pred_label += f"\n({pred_class[:15]})"
                
                axes[i].set_title(
                    f"{true_label}\n{pred_label}",
                    fontsize=9,
                    color='red',
                    fontweight='bold'
                )
                axes[i].axis('off')
        
        # 빈 subplot 숨기기
        for i in range(n_show, 12):
            axes[i].axis('off')
        
        plt.tight_layout()
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.show()
        print(f"\n💾 틀린 이미지 저장: {save_path}")

print("✅ analyze_wrong_predictions 함수 정의 완료")


In [ ]:
# meta.csv 로드
meta_df = pd.read_csv(f"{CFG['BASE_DIR']}/data/meta.csv")

# Wrong Predictions 분석
analyze_wrong_predictions(
    val_dataset.df,
    val_preds,
    val_targets,
    CFG['TRAIN_IMG_DIR'],
    meta_df=meta_df,
    save_path=f"{CFG['BASE_DIR']}/wrong_predictions.png"
)


## Confusion Matrix


In [ ]:
# Confusion Matrix
cm = confusion_matrix(val_targets, val_preds)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')

# 축 레이블 (클래스 이름)
tick_labels = [f"{i}\n{meta_df[meta_df['target']==i]['class_name'].values[0][:10]}" 
               for i in range(CFG['NUM_CLASSES'])]
plt.xticks(np.arange(CFG['NUM_CLASSES']) + 0.5, tick_labels, rotation=45, ha='right', fontsize=8)
plt.yticks(np.arange(CFG['NUM_CLASSES']) + 0.5, tick_labels, fontsize=8)

plt.tight_layout()
plt.savefig(f"{CFG['BASE_DIR']}/confusion_matrix.png", dpi=150, bbox_inches='tight')
plt.show()



## Prediction Distribution


In [ ]:
# 클래스별 예측 분포
print(f"\n📈 Validation 클래스별 예측 분포:")
val_pred_counts = pd.Series(val_preds).value_counts().sort_index()
for class_id, count in val_pred_counts.items():
    class_name = meta_df[meta_df['target'] == class_id]['class_name'].values[0]
    print(f"   Class {class_id:2d} ({class_name[:25]:25s}): {count:3d}개")


## Inference & Submission


In [ ]:
# Best 모델 로드
model.load_state_dict(torch.load(f"{CFG['BASE_DIR']}/06_best_model.pth"))
model.eval()

# Test 예측
preds_list = []
for imgs, _ in tqdm(tst_loader, desc="Inference"):
    imgs = imgs.to(CFG['DEVICE'])
    outputs = model(imgs)
    preds_list.extend(outputs.argmax(dim=1).cpu().numpy())

# 제출 파일 생성
submission = pd.read_csv(CFG['TEST_CSV'])
submission['target'] = preds_list
submission.to_csv(f"{CFG['BASE_DIR']}/submission.csv", index=False)

